# Resolved-rate control of the end-effector

The goal of this and the next laboratory is to implement a resolved-rate controller to control the end-effector. You should aim to at least finish Part 1 during Lab 5 and Part 2 during Lab 6. We recommend that you read all the instructions in Part 2 well before Lab 6.

Last laboratory, we used analytic IK to compute desired positions for the joint angles that would ensure we reach a desired position (touching the pink balls). Then we generated a straight line trajectory in joint space and used the joint PD controller to track this trajectory.

Here, we will use the fact that $$\mathcal{V}_{SO}^O = J_O(\boldsymbol{\theta}) \cdot \dot{\boldsymbol{\theta}}$$ 
to derive directly a controller acting in the end-effector space,
where $\boldsymbol{\theta} = [\theta_0, \theta_1, \theta_2]$ is the vector containing all the joint positions and $J_O$ is the Jacobian used to compute twists in Frame 0.

## Instructions
Answer all the questions below and submit a pdf with detailed answers to these questions, including the plots through Brightspace. You will also need to submit the Jupyter notebooks with the code used to answer the questions.

## Robot model
Consider the following kinematic model of the robot <img src="./3d_robot_model.png" width="700">


In [1]:
#setup nice plotting
%matplotlib notebook

# we import useful libraries
import time
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

# we import the robot class
use_real_robot = True

if use_real_robot:
    from nyu_finger import NYUFingerReal    
else:
    from nyu_finger_simulator import NYUFingerSimulator
    
    
# here we define the global variables for the robot size
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

# Part 1: compute the end-effector Jacobian

1. Write a ``compute_jacobian`` function which computes the Jacobian of the end-effector. The computed Jacobian should be with respect to a frame located at the end-effector but oriented like the spatial frame (Frame O in the figure above). You will need to reuse the kinematics functions written in the previous laboratories.

Here make sure to return a 6x3 matrix (you will take the rows corresponding to the linear velocity in the following questions).

2. To test your Jacobian function, use your code from Lab 4 that makes the robot reach 3 different goals and compute the velocity of the end-effector at each instant of time. Compare the computed velocities with the velocities you get by numerically differentiating the end-effector positions (i.e. by computing $v[t] = \frac{x[t] - x[t-1]}{\Delta t}$). 

Here you will ignore the angular velocity and only consider the linear velocity of the end-effector. If we only look at the linear velocity of point O (i.e. the velocity of the end-effector with respect to frame O), we can decompose the Jacobian into two part (the top 3 rows are associated to the angular velocity of the end-effector and the bottom 3 are associated to the linear velocity): $$J_O(\boldsymbol{\theta}) = \begin{bmatrix} J_{O\omega} \\ J_{Ov} \end{bmatrix}$$
and we can write the velocity equation as
$$\mathcal{V}_{SO}^O = \begin{pmatrix} \omega_O \\ v_O \end{pmatrix} = \begin{bmatrix} J_{O\omega} \\ J_{Ov} \end{bmatrix} \cdot \dot{\boldsymbol{\theta}}$$

So the linear velocity of the end-effector is $$v_O = J_{Ov} \dot{\boldsymbol{\theta}}$$


3. Plot the velocities computed in question 2.

In [2]:
def translate(vector): # vector: np.array([1,2,3])
    transform = np.eye(4)
    transform[:3,3:] = vector.reshape(3,1)
    return transform

def rotateX(angle):
    transform = np.eye(4)
    transform[1:3, 1:3] = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    return transform

def rotateY(angle):
    transform = np.eye(4)
    transform[:3, :3] = np.array([[np.cos(angle), 0, np.sin(angle)], [0,1,0], [-np.sin(angle), 0, np.cos(angle)]])
    return transform

def rotateZ(angle):
    transform = np.eye(4)
    transform[:2, :2] = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    return transform

# test
# print(translate(np.array([1,2,3])).dot(rotateX(PI/6)))

def forward_kinematics(theta):
    # assume theta is a vector, theta[0] = theta0, theta[1] = theta1, etc.
    # we define the lengths of the robot
#     l0 = 0.3
#     l1 = 0.16
#     l2 = 0.16
#     l3 = 0.014

    T_SH1 = translate(np.array([l0, 0, 0]))
    T_H1H2 = rotateX(theta[0]) @ translate(np.array([0, 0, l3])) @ rotateZ(theta[1])
    T_H2K = translate(np.array([0, -l1, 0])) @ rotateZ(theta[2])
    T_KF = translate(np.array([0, -l2, 0]))
    
    T_SF = T_SH1 @ T_H1H2 @ T_H2K @ T_KF
    
    return T_SF

def adjoint(T):
    AdT = np.zeros((6,6))
    R = T[:3, :3]
    p = T[:3, 3]
    AdT[:3, :3] = R
    AdT[3:6, 3:6] = R
    AdT[3:6, :3] = vec2skewsym(p) @ R
    return AdT

def vec2skewsym(array):
    a_skew = np.zeros((3,3))
    a_skew[2][1], a_skew[1][2] = array[0], -array[0]
    a_skew[0][2], a_skew[2][0] = array[1], -array[1]
    a_skew[1][0], a_skew[0][1] = array[2], -array[2]
    return a_skew

def get_only_IK(x,y):
    '''
    Input: (x,y) position of the fingertip
    Output: one of theta answers (max 1,3)
    '''
    # Use the first solution, account for no solution
    q_theta = inverse_kinematics(x,y)
    if len(q_theta) != 0:
        return np.array(q_theta[0])
    
def inverse_kinematics(x,y):
    """
    inverse kinematics function
    input (x,y) position of the fingertip
    output a list of 3D vectors which are possible solutions to the problem 
    (the list is empty if there are no solutions)
    """
    sol = []
    l_sq = (x - l0) ** 2 + y ** 2
    l_max_sq = (l1 + l2) ** 2
    beta = np.arctan2(y,x-l0)
    theta1 = np.pi/2 + beta
    
    if l_sq > l_max_sq:
        pass
    
    elif l_sq == l_max_sq:
        sol.append([0, theta1, 0])
        
    else:
        alpha = np.arccos((l_sq + l1**2 - l2**2) / (2 * l1 * np.sqrt(l_sq)))
        theta1_p = theta1 - alpha
        theta2_p = np.arccos((l_sq - l1**2 - l2**2) / (2 * l1 * l2))
        sol.append([0, theta1_p, theta2_p])
        
        theta1_n = theta1 + alpha
        theta2_n = - theta2_p
        sol.append([0, theta1_n, theta2_n])
    
    return sol
    # return a list of solutions [[theta0, theta1, theta2]]
    
def compute_trajectory(position_init, position_goal, t_init, t_goal, t):
    T = t_goal - t_init
    dt = t - t_init
    dpos = position_goal - position_init
    pos_param = 10 * (dt / T) ** 3 - 15 * (dt / T) ** 4 + 6 * (dt / T) ** 5
    vel_param = (30 * (dt / T) ** 2 - 60 * (dt / T) ** 3 + 30 * (dt / T) ** 4) / T
    desired_position = position_init + pos_param * dpos
    desired_velocity = vel_param * dpos
    
    # we return the answer in joint angles
    return desired_position, desired_velocity

In [3]:
def compute_jacobian(theta):
    T_SH1 = translate(np.array([l0, 0, 0]))
    T_H1H2 = rotateX(theta[0]) @ translate(np.array([0, 0, l3])) @ rotateZ(theta[1])
    T_H2K = translate(np.array([0, -l1, 0])) @ rotateZ(theta[2])
    T_KF = translate(np.array([0, -l2, 0]))
    
    
    J = np.zeros([3,3])
    
    w_H1 = np.array([[1,0,0,0,0,0]])
    J1_S = adjoint(T_SH1) @ w_H1.T

    
    w_H2 = np.array([[0,0,1,0,0,0]])
    J2_S = adjoint(T_SH1 @ T_H1H2) @ w_H2.T
 
    
    w_K = np.array([[0,0,1,0,0,0]])
    J3_S = adjoint(T_SH1 @ T_H1H2 @ T_H2K) @ w_K.T
    
    J_S = np.hstack([J1_S, J2_S, J3_S])
    
    T_SO = forward_kinematics(theta)
    T_SO[:3, :3] = np.zeros(3)
    T_OS = np.eye(4)
    T_OS[:3, 3] = -T_SO[:3, 3]
#     print(T_OS)
    J = adjoint(T_OS) @ J_S
    
    return J[3:,:]

compute_jacobian([0,0,0])



array([[ 0.   ,  0.32 ,  0.16 ],
       [-0.014,  0.   ,  0.   ],
       [-0.32 ,  0.   ,  0.   ]])

In [4]:
# if use_real_robot:
#     # Triggers the real robot
#     # Don't forget to turn on the robot first !!!
#     robot = NYUFingerReal()
#     # Make sure that the motor number matches that on the robot
#     motor_number = np.array([3,2,0])
#     robot.initialize('ens1', motor_number)
# else:
#     # we can now create a robot simulation
#     robot = NYUFingerSimulator()

# # we reset the simulation
# if not use_real_robot:
#     robot.reset_state([0,0,0])

    
# # we simulate for 15 seconds (5 seconds per goal)
# run_time = 15.
# dt = 0.001
# num_steps = int(run_time/dt)

# # the PD gains
# P = np.array([4,4,4])
# D = np.array([0.05, 0.05, 0.05])

# ## we allocate a few vectors to collect data while we do the simulation
# # we will store the sensor values (angular position and velocities)
# measured_positions = np.zeros([num_steps,3])
# measured_velocities = np.zeros_like(measured_positions)
# # we will store the commands sent to the robot
# desired_torques = np.zeros_like(measured_positions)
# # we will store the desired joint trajectories that the PD controller follows
# desired_positions = np.zeros_like(measured_positions)
# desired_velocities = np.zeros_like(measured_positions)
# # we keep track of time
# time = np.zeros([num_steps])
# # we will store the end effector position (desired and measured)
# measured_finger_position = np.zeros([num_steps,3])
# desired_finger_position = np.zeros([num_steps,3])

# measured_linear_velocity = np.zeros([num_steps,3])
# computed_linear_velocity = np.zeros([num_steps,3])

# # here we create a list of ball positions and display them
# ball_positions = [np.array([0.597,-0.056]), np.array([0.521,0.12]), np.array([0.3,-0.225])]
# if not use_real_robot:
#     for ball in ball_positions:
#         robot.add_ball(ball[0], ball[1])
# else:
#     # we wait
#     for i in range(7500):
#         robot.send_joint_torque(np.zeros([3,1]))
#         robot.step()

# # the time to reach a goal
# time_to_goal = 5.0

# # we use this to know which target we are currently aiming
# ball_number = 0

# # we measure the internal time for one target movement
# t = 0.

# '''Initialize all goals'''
# start_pos = np.array([l0, -(l1 + l2)])
# goals = [start_pos] + ball_positions


# '''Lab5'''

# # Select either fifth interpolation or linear interpolation
# linear_interp = False

# for i in range(num_steps):
#     # get the current time and save it
#     time[i] = dt * i
    
#     # we read the position and velocities of the joints from the robot or simulation
#     q, dq = robot.get_state()
    
#     # we store these values for later use
#     measured_positions[i,:] = q
#     measured_velocities[i,:] = dq
    
#     # save the current position of the fingertip using the FK function
#     pose = forward_kinematics(q)
#     measured_finger_position[i,:] = pose[0:3,3]
    
#     measured_linear_velocity[i,:] = (measured_finger_position[i,:] - measured_finger_position[i-1,:])/dt
#     computed_linear_velocity[i,:] = compute_jacobian(q) @ dq.T
    
# #     print(measured_linear_velocity - computed_linear_velocity)
    
#     # controller: TODO HERE IMPLEMENT YOUR CONTROLLER TO REACH ALL 3 TARGETS
#     # the goal here is to compute q_des and dq_des that the PD controller will follow
#     # you will need some internal logic to decide which ball you are currently targetting, etc.

#     '''User code starts'''
    
#     # Identify the start and end coordinates for each path
#     cur_path = int(time[i] // time_to_goal)
#     start_coord = goals[cur_path]
#     end_coord = goals[cur_path+1]
    
#     t_init = cur_path * time_to_goal
#     t_goal = (cur_path + 1) * time_to_goal
    
#     start_theta = get_only_IK(start_coord[0], start_coord[1])
#     end_theta = get_only_IK(end_coord[0], end_coord[1])
    
#     if linear_interp:
#         dq_des = (end_theta - start_theta) / time_to_goal
#         q_des = start_theta + (time[i] - t_init) * dq_des
#     else:
#         q_des, dq_des = compute_trajectory(start_theta, end_theta, t_init, t_goal, time[i])
        
#     '''User code ends'''
    
#     # we save the desired positions/velocities for later plotting 
#     desired_positions[i,:] = q_des
#     desired_velocities[i,:] = dq_des
    
#     ##PD controller
#     error = q_des - q # the position error for all the joints (it's a 3D vector)
#     d_error = dq_des-dq # the velocity error for all the joints
    
#     # we compute the desired torques as a PD controller
#     joint_torques = P * error + D * d_error
#     desired_torques[i,:] = joint_torques
    
#     # we send them to the robot and do one simulation step
#     robot.send_joint_torque(joint_torques)
#     robot.step()

In [5]:
# def plot_finger_velocity(measured_linear_velocity, computed_linear_velocity):
#     fig = plt.figure()
  
#     plt.subplot(3,1,1)
#     plt.plot(time[1:], measured_linear_velocity[1:,0], 'b-', time[1:], computed_linear_velocity[1:,0], '--k')
#     plt.ylabel(r'$x$')
# #     plt.ylim([-0.5,0.5])
    
#     plt.subplot(3,1,2)
#     plt.plot(time[1:], measured_linear_velocity[1:,1], 'b-', time[1:], computed_linear_velocity[1:,1], '--k')
#     plt.ylabel(r'$y$')
# #     plt.ylim([-0.5,0.5])

#     plt.subplot(3,1,3)
#     plt.plot(time[1:], measured_linear_velocity[1:,2], 'b-', time[1:], computed_linear_velocity[1:,2], '--k')
#     plt.ylabel(r'$z$')
#     plt.xlabel('Time [s]')
# #     plt.ylim([-0.5,0.5])
    
    
# plot_finger_velocity(measured_linear_velocity, computed_linear_velocity)

# Part 2: resolved rate control

## End-effector velocities

In contrast to Lab 4 where we generated trajectories in "joint space", i.e. we had a desired start and goal position for each joint, here we will directly generate trajectories for the end-effector and follow them without ever explicitly computing a desired joint position goal. We will work in "end-effector space".

Because our robot have 3 joints, away from singular configurations, the matrix $J_{Ov}$ (which is 3x3) will be invertible. It means that the joint velocity that achieves a desired end-effector velocity is
$$ \dot{\boldsymbol{\theta}} = J_{Ov}^{-1} v_O$$

## End-effector control

If we have a desired velocity we would like to achieve for the end-effector, we can then directly convert it into a desired velocity for the joints. Then we could use our joint PD controller where we would remove the P part to track this joint space velocity.

Assume we have a reference end-effector trajectory (e.g. coming from our trajectory generator that create smooth trajectories from a starting point to a desired goal with 0 velocities and accelerations at the beginning and end of the trajectories). At each instant of time, we then get $x_{ref}$ and $\dot{x}_{ref}$, the desired position and velocity of the end-effector. Our desired end-effector velocity (which is different from the target) could then be a combination between the reference velocity and an error correction term to make sure we stay on the trajectory
$$v_O = \dot{x}_{ref} + P (x_{ref} - x_{measured})$$

and so the desired joint velocities would be
$$\dot{\boldsymbol{\theta}}_{des} = J_{Ov}^{-1} \left( \dot{x}_{ref} + P (x_{ref} - x_{measured}) \right)$$

and the command sent to the robot
$$\tau = D(\dot{\boldsymbol{\theta}}_{des} - \dot{\boldsymbol{\theta}}_{measured})$$

where D is the gain we used in our previous lab for the PD control (the P in the PD controller being canceled).

## Dealing with singularities (very important!!)

The Jacobian becomes singular, i.e. non invertible, when $\theta_2 = 0$. In this case, our control law will not be well defined because the Jacobian cannot be inverted (we will see in class how to deal with this situation too later).

For now, there are 2 simple ways to handle this situation:
1. make sure the robot never reaches a singular configuration, whenever $|\theta_2| < 0.05$ stop everything,
or,
2. when $|\theta_2| < 0.05$ (i.e. we are close to a singularity), we can change the matrix to inverse to make it invertible by replacing $J_{Ov}^{-1}$ by $(J_{Ov} + \epsilon I)^{-1}$ where $\epsilon$ is a small number, e.g. 10e-4.

## Question 1: goal-reaching
1. Write a resolved-rate controller that solves the goal-reaching tasks from Lab 3 and 4 - generate end-effector trajectories using a 5th order polynomial as in Lab 4.
2. Compare the results with the results from Lab 4 - how does the end-effector trajectory change? Plot the measured end-effector position and velocity of the end-effector and the reference ones. The quantities should be graphed vs. time (use the Jacobian to get the measured end-effector velocities)
3. Run the experiment on the real robot and analyze the results.

## Question 2: other movements
1. Write a controller that can draw a circle in the x-y plane. We would like to follow the 3D end-effector trajectory $$x_{ref}(t) = \begin{pmatrix} 0.3 + 0.05\cos(\omega t) \\ -0.2 + 0.05 \sin(\omega t) \\ 0 \end{pmatrix}$$
where $\omega = \pi$ is the frequency of rotation around the circle (frequency of rotation of $0.5Hz$).
2. Plot the measured end-effector position and velocity of the end-effector and the reference ones. The quantities should be graphed vs. time (use the Jacobian to get the measured end-effector velocities)
3. Test the controller(s) on the real robot and plot the same plots as for the simulation
4. Now write a controller that can do a "figure 8" in the x-y plan $$x_{ref}(t) = \begin{pmatrix} 0.35 + 0.08\sin(\omega t) \\ -0.15 + 0.08 \cos(\omega t) \sin(\omega t) \\ 0 \end{pmatrix}$$. Show the same plots as in the previous questions.
5. Write a controller that can do a circle in the x-z plane.

In [ ]:
if use_real_robot:
    # Triggers the real robot
    # Don't forget to turn on the robot first !!!
    robot = NYUFingerReal()
    # Make sure that the motor number matches that on the robot
    motor_number = np.array([3,2,0])
    robot.initialize('ens1', motor_number)
else:
    # we can now create a robot simulation
    robot = NYUFingerSimulator()

# we reset the simulation
if not use_real_robot:
    robot.reset_state([0,-.05,0.05])

    
# we simulate for 15 seconds (5 seconds per goal)
run_time = 15.
dt = 0.001
num_steps = int(run_time/dt)

# the PD gains
P = np.array([4,4,4])
D = np.array([0.05, 0.05, 0.05])

## we allocate a few vectors to collect data while we do the simulation
# we will store the sensor values (angular position and velocities)
measured_positions = np.zeros([num_steps,3])
measured_velocities = np.zeros_like(measured_positions)
# we will store the commands sent to the robot
desired_torques = np.zeros_like(measured_positions)
# we will store the desired joint trajectories that the PD controller follows
desired_positions = np.zeros_like(measured_positions)
desired_velocities = np.zeros_like(measured_positions)
# we keep track of time
time = np.zeros([num_steps])
# we will store the end effector position (desired and measured)
measured_finger_position = np.zeros([num_steps,3])
desired_finger_position = np.zeros([num_steps,3])

measured_linear_velocity = np.zeros([num_steps,3])
computed_linear_velocity = np.zeros([num_steps,3])

# here we create a list of ball positions and display them
ball_positions = [np.array([0.597,-0.056]), np.array([0.521,0.12]), np.array([0.3,-0.225])]
if not use_real_robot:
    for ball in ball_positions:
        robot.add_ball(ball[0], ball[1])
else:
    # we wait
    for i in range(7500):
        robot.send_joint_torque(np.zeros([3,1]))
        robot.step()

# the time to reach a goal
time_to_goal = 5.0

# we use this to know which target we are currently aiming
ball_number = 0

# we measure the internal time for one target movement
t = 0.

'''Initialize all goals'''
start_pos = np.array([l0, -(l1 + l2)])
goals = [start_pos] + ball_positions


'''Lab4'''

# Select either fifth interpolation or linear interpolation
linear_interp = False

for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we read the position and velocities of the joints from the robot or simulation
    q, dq = robot.get_state()
    
    # we store these values for later use
    measured_positions[i,:] = q
    measured_velocities[i,:] = dq
    
    # save the current position of the fingertip using the FK function
    pose = forward_kinematics(q)
    measured_finger_position[i,:] = pose[0:3,3]
    
    ''''''
    jacob = compute_jacobian(q)
    measured_linear_velocity[i,:] = (measured_finger_position[i,:] - measured_finger_position[i-1,:])/dt
    computed_linear_velocity[i,:] = jacob @ dq.T
    
#     print(measured_linear_velocity - computed_linear_velocity)
    
    # controller: TODO HERE IMPLEMENT YOUR CONTROLLER TO REACH ALL 3 TARGETS
    # the goal here is to compute q_des and dq_des that the PD controller will follow
    # you will need some internal logic to decide which ball you are currently targetting, etc.

    '''User code starts'''
    
    # Identify the start and end coordinates for each path
    cur_path = int(time[i] // time_to_goal)
    start_coord = goals[cur_path]
    start_coord = np.append(start_coord, l3)
#     print(start_coord)
    end_coord = goals[cur_path+1]
    end_coord = np.append(end_coord, l3)
    
    t_init = cur_path * time_to_goal
    t_goal = (cur_path + 1) * time_to_goal
    
    start_theta = get_only_IK(start_coord[0], start_coord[1])
    end_theta = get_only_IK(end_coord[0], end_coord[1])
    
    if linear_interp:
        dq_des = (end_theta - start_theta) / time_to_goal
        q_des = start_theta + (time[i] - t_init) * dq_des
    else:
        
        x_ref, dx_ref = compute_trajectory(start_coord, end_coord, t_init, t_goal, time[i])
        
        dx_des = dx_ref + P * (x_ref - measured_finger_position[i,:])
        print(x_ref - measured_finger_position[i,:])
        
        if abs(q[2]) < 0.05:
            inv_jacob = np.linalg.inv(jacob + 1e-4 * np.eye(3))
        else:
            inv_jacob = np.linalg.inv(jacob)
        
        dq_des = inv_jacob @ dx_des
        q_des = get_only_IK(x_ref[0], x_ref[1])
        
        
        
        
    '''User code ends'''
    
    # we save the desired positions/velocities for later plotting 
    desired_positions[i,:] = q_des
    desired_velocities[i,:] = dq_des
    
    ##PD controller
    error = q_des - q # the position error for all the joints (it's a 3D vector)
    d_error = dq_des-dq # the velocity error for all the joints
    
    # we compute the desired torques as a PD controller
    ''''''
#     joint_torques = P * error + D * d_error
    joint_torques = D * d_error
    desired_torques[i,:] = joint_torques
    
    # we send them to the robot and do one simulation step
    robot.send_joint_torque(joint_torques)
    robot.step()

[ 5.00101428e-04 -3.73981295e-05 -8.20605036e-04]
[ 5.00101452e-04 -3.73981084e-05 -8.20605036e-04]
[ 5.00101618e-04 -3.73979607e-05 -8.20605036e-04]
[ 5.00102069e-04 -3.73975598e-05 -8.20605036e-04]
[ 5.00102947e-04 -3.73967795e-05 -8.20605036e-04]
[ 5.00104393e-04 -3.73954935e-05 -8.20605036e-04]
[ 5.00106551e-04 -3.73935758e-05 -8.20605036e-04]
[ 5.00109561e-04 -3.73909006e-05 -8.20605036e-04]
[ 5.00113564e-04 -3.73873420e-05 -8.20605036e-04]
[ 5.00118702e-04 -3.73827746e-05 -8.20605036e-04]
[ 5.00125117e-04 -3.73770729e-05 -8.20605036e-04]
[ 5.00132948e-04 -3.73701115e-05 -8.20605036e-04]
[ 5.00142338e-04 -3.73617654e-05 -8.20605036e-04]
[ 5.00153425e-04 -3.73519097e-05 -8.20605036e-04]
[ 5.00166352e-04 -3.73404194e-05 -8.20605036e-04]
[ 5.00181258e-04 -3.73271699e-05 -8.20605036e-04]
[ 5.00198282e-04 -3.73120367e-05 -8.20605036e-04]
[ 5.00217566e-04 -3.72948955e-05 -8.20605036e-04]
[ 5.00239249e-04 -3.72756219e-05 -8.20605036e-04]
[ 5.05842783e-04 -3.72660744e-05 -8.20605005e-04]


[ 3.75517865e-03  9.58437761e-05 -8.20508344e-04]
[ 3.73564660e-03  9.86707800e-05 -8.20509285e-04]
[ 0.00373844  0.00010115 -0.00082051]
[ 0.00374126  0.00010366 -0.00082051]
[ 0.00373295  0.00010631 -0.00082051]
[ 0.00372466  0.00010899 -0.00082051]
[ 0.00373313  0.00011196 -0.00080937]
[ 0.00374721  0.00011432 -0.00080937]
[ 0.00376132  0.0001167  -0.00080937]
[ 0.0037643   0.00011934 -0.00080937]
[ 0.00378961  0.00012153 -0.00080936]
[ 0.0038038   0.00012397 -0.00080936]
[ 0.00382359  0.00012632 -0.00080936]
[ 0.00382668  0.00012906 -0.00080936]
[ 0.00382979  0.00013171 -0.00080936]
[ 0.0038385   0.00013374 -0.0008205 ]
[ 0.00382493  0.00013668 -0.0008205 ]
[ 0.00383371  0.00013939 -0.0008205 ]
[ 0.00383135  0.00014226 -0.0008205 ]
[ 0.00382344  0.00014527 -0.0008205 ]
[ 0.00382114  0.00014819 -0.0008205 ]
[ 0.00382444  0.00015112 -0.0008205 ]
[ 0.00383335  0.00015396 -0.0008205 ]
[ 0.00384229  0.00015683 -0.0008205 ]
[ 0.00384568  0.00016035 -0.00080936]
[ 0.0038491   0.00016327 -

[ 0.01213526  0.00289528 -0.000829  ]
[ 0.01191117  0.00291935 -0.00082902]
[ 0.01184255  0.00292668 -0.00082899]
[ 0.01173509  0.00293615 -0.00082896]
[ 0.01175585  0.00295512 -0.00081786]
[ 0.01168732  0.00295795 -0.00081782]
[ 0.01156893  0.00296755 -0.00081779]
[ 0.01156175  0.00297199 -0.00081776]
[ 0.01141595  0.00299423 -0.00081777]
[ 0.01154228  0.00299399 -0.00081772]
[ 0.01181866  0.0029862  -0.00081765]
[ 0.01205617  0.0029789  -0.00082869]
[ 0.01233813  0.00296984 -0.00082861]
[ 0.01238164  0.00298824 -0.00082861]
[ 0.01264711  0.00298296 -0.00083967]
[ 0.01286262  0.00298062 -0.00083961]
[ 0.01307818  0.00297905 -0.00083955]
[ 0.01305526  0.00299932 -0.00083956]
[ 0.01338732  0.0029835  -0.00083947]
[ 0.01354736  0.00298012 -0.00083941]
[ 0.01366853  0.00297767 -0.00082825]
[ 0.01355695  0.00300117 -0.00081715]
[ 0.01378382  0.00299726 -0.00080598]
[ 0.01382186  0.00299792 -0.00080593]
[ 0.01377675  0.00300531 -0.0008059 ]
[ 0.01372616  0.00301331 -0.00080587]
[ 0.01365893

[ 0.0536434   0.00553419 -0.00083675]
[ 0.0538683   0.00550656 -0.00083656]
[ 0.05391539  0.00554842 -0.00083656]
[ 0.05418724  0.005501   -0.00083632]
[ 0.05446425  0.00545133 -0.00083607]
[ 0.05473072  0.00540517 -0.00083583]
[ 0.05499219  0.0053632  -0.0008356 ]
[ 0.05503963  0.00540536 -0.0008356 ]
[ 0.05533281  0.00535415 -0.00083535]
[ 0.05567788  0.0052805  -0.00083504]
[ 0.05599174  0.00522064 -0.00083476]
[ 0.05628983  0.00516626 -0.0008345 ]
[ 0.0563376   0.00520872 -0.0008345 ]
[ 0.05659882  0.00516521 -0.00083427]
[ 0.0568753   0.00511284 -0.00083401]
[ 0.05709924  0.00507739 -0.0008338 ]
[ 0.05731761  0.00504107 -0.00083358]
[ 0.0575467   0.0050032  -0.00083337]
[ 0.05759488  0.00504604 -0.00083337]
[ 0.05781292  0.0050067  -0.00083314]
[ 0.05797924  0.00498998 -0.00083298]
[ 0.05811991  0.00498596 -0.00083285]
[ 0.05826061  0.00498191 -0.00083273]
[ 0.05830914  0.00502505 -0.00083273]
[ 0.05845548  0.00502169 -0.0008326 ]
[ 0.05861294  0.00501963 -0.00083248]
[ 0.05879122

[ 0.0789101   0.0099961  -0.00284616]
[ 0.07893334  0.01006829 -0.00284622]
[ 0.07899576  0.01007517 -0.00293729]
[ 0.07900358  0.01010219 -0.0029878 ]
[ 0.07900076  0.01012727 -0.00301792]
[ 0.07900231  0.01014695 -0.00301744]
[ 0.07907677  0.01021314 -0.00301744]
[ 0.07911166  0.0102106  -0.00298622]
[ 0.07920659  0.01019295 -0.00293449]
[ 0.07935985  0.01014579 -0.0028825 ]
[ 0.07949812  0.01010194 -0.00284075]
[ 0.07957289  0.0101684  -0.00284075]
[ 0.07971601  0.01012103 -0.00280916]
[ 0.0798201   0.01009    -0.00280822]
[ 0.07989801  0.0100542  -0.00282753]
[ 0.07996581  0.01001834 -0.00286713]
[ 0.08004089  0.01008509 -0.00286713]
[ 0.08010658  0.01003024 -0.00292681]
[ 0.0801734   0.00998467 -0.00297639]
[ 0.08023067  0.00994523 -0.00301586]
[ 0.08033496  0.00991176 -0.00304523]
[ 0.08041035  0.00997818 -0.00305536]
[ 0.08058453  0.00992871 -0.00305413]
[ 0.08077414  0.00988001 -0.0030428 ]
[ 0.08103342  0.00981448 -0.00301104]
[ 0.0810787   0.0098873  -0.00298073]
[ 0.08145536

...

[ 0.11826959  0.24315302 -0.03397617]
[ 0.11845297  0.24331081 -0.03401158]
[ 0.11854004  0.24340268 -0.03402996]
[ 0.11875696  0.24359022 -0.03407309]
[ 0.11884905  0.24368533 -0.03409233]
[ 0.11894122  0.24377866 -0.0341161 ]
[ 0.11902863  0.24387001 -0.03413438]
[ 0.11901974  0.24389058 -0.03413438]
[ 0.11909758  0.24397471 -0.03415083]
[ 0.1192233   0.24410112 -0.03417324]
[ 0.11929166  0.24417801 -0.03418783]
[ 0.11928284  0.24419843 -0.03418783]
[ 0.11938988  0.24430505 -0.03420514]
[ 0.11947283  0.24439225 -0.03422242]
[ 0.11946406  0.24441255 -0.03422242]
[ 0.11945531  0.2444328  -0.03422242]
[ 0.11955292  0.24453041 -0.03424242]
[ 0.11965539  0.24463138 -0.03426331]
[ 0.11964669  0.24465152 -0.03426331]
[ 0.11974443  0.24475051 -0.0342787 ]
[ 0.11986201  0.24485631 -0.03430086]
[ 0.11985336  0.24487634 -0.03430086]
[ 0.11984473  0.24489632 -0.03430086]
[ 0.12000084  0.24503698 -0.03432705]
[ 0.11999224  0.24505689 -0.03432705]
[ 0.11998366  0.24507676 -0.03432705]
[ 0.12007689

[ 0.14689162  0.26703827 -0.03867568]
[ 0.14688874  0.26704495 -0.03867568]
[ 0.14694593  0.26707928 -0.03868283]
[ 0.14701403  0.26711858 -0.03869126]
[ 0.14707671  0.26715531 -0.03869904]
[ 0.14715513  0.26719416 -0.03870737]
[ 0.14715231  0.2672007  -0.03870737]
[ 0.14723067  0.26723944 -0.03871569]
[ 0.14732079  0.26728849 -0.03872668]
[ 0.14741094  0.26733746 -0.03873765]
[ 0.14752297  0.26739631 -0.03875119]
[ 0.14752021  0.26740088 -0.03875635]
[ 0.14763779  0.26746209 -0.03877051]
[ 0.14776635  0.26752816 -0.03878593]
[ 0.14788953  0.26759161 -0.03880068]
[ 0.14802371  0.2676599  -0.03881668]
[ 0.14801629  0.26766903 -0.03881742]
[ 0.14812939  0.26773262 -0.03883223]
[ 0.14824178  0.26779073 -0.03884563]
[ 0.14832761  0.26784359 -0.03885202]
[ 0.14839625  0.26788181 -0.03886029]
[ 0.14845941  0.2679175  -0.03886792]
[ 0.14845678  0.2679236  -0.03886792]
[ 0.14851917  0.26795387 -0.03887417]
[ 0.1485824   0.26798942 -0.03888178]
[ 0.14864565  0.26802493 -0.03888939]
[ 0.14871915

[ 0.17516876  0.27841669 -0.04128936]
[ 0.17532035  0.27845689 -0.04129972]
[ 0.17532034  0.27845693 -0.04129972]
[ 0.17540783  0.27847815 -0.0413112 ]
[ 0.17540781  0.27847817 -0.0413112 ]
[ 0.17550113  0.2785028  -0.04131754]
[ 0.17559446  0.27852738 -0.04132387]
[ 0.17570532  0.2785565  -0.04133137]
[ 0.17580452  0.27858249 -0.04133806]
[ 0.17590959  0.27860995 -0.04134514]
[ 0.17590958  0.27860997 -0.04134514]
[ 0.17601637  0.27864248 -0.04135352]
[ 0.17612148  0.27866788 -0.04136609]
[ 0.17620907  0.2786906  -0.04137194]
[ 0.17629084  0.27871176 -0.0413774 ]
[ 0.17629083  0.27871177 -0.0413774 ]
[ 0.17635509  0.27872837 -0.04138168]
[ 0.17642519  0.27874646 -0.04138634]
[ 0.17648362  0.2787615  -0.04139022]
[ 0.17653621  0.27877503 -0.0413937 ]
[ 0.17653621  0.27877503 -0.0413937 ]
[ 0.17659463  0.27879004 -0.04139757]
[ 0.17665892  0.27880652 -0.04140182]
[ 0.17672906  0.27882254 -0.04141198]
[ 0.17679921  0.27884047 -0.04141661]
[ 0.17679921  0.27884047 -0.04141661]
[ 0.17688105

[ 0.19951904  0.28190136 -0.04247102]
[ 0.19961464  0.28190309 -0.04247406]
[ 0.19960815  0.28189296 -0.04247406]
[ 0.19981174  0.28190692 -0.04248028]
[ 0.19991927  0.28190969 -0.04248363]
[ 0.20000275  0.28190963 -0.04248626]
[ 0.19999613  0.28189929 -0.04248626]
[ 0.20007956  0.28189908 -0.04248887]
[ 0.20014494  0.28189675 -0.04249096]
[ 0.20020429  0.28189367 -0.04249287]
[ 0.2002456   0.28188851 -0.04249425]
[ 0.20028688  0.28188329 -0.04249563]
[ 0.2003221   0.28187733 -0.04249683]
[ 0.20036332  0.28187198 -0.0424982 ]
[ 0.20035644  0.28186125 -0.0424982 ]
[ 0.20039759  0.28185382 -0.04250526]
[ 0.2004387   0.28184829 -0.04250662]
[ 0.20049781  0.28184468 -0.0425085 ]
[ 0.20055088  0.28184034 -0.04251019]
[ 0.20060392  0.28183592 -0.04251189]
[ 0.20059685  0.28182489 -0.04251189]
[ 0.20066185  0.28182166 -0.04251391]
[ 0.20072681  0.28181835 -0.04251592]
[ 0.20079775  0.28181561 -0.0425181 ]
[ 0.20087467  0.28181343 -0.04252043]
[ 0.20096358  0.28181244 -0.04252308]
[ 0.20095631

[ 0.216566    0.27408935 -0.04273708]
[ 0.21654285  0.2740532  -0.04273708]
[ 0.2165801   0.27401623 -0.04273689]
[ 0.21661731  0.27397917 -0.04273669]
[ 0.21666049  0.27394193 -0.04273647]
[ 0.21671574  0.27390442 -0.0427362 ]
[ 0.21669234  0.2738679  -0.0427362 ]
[ 0.2167475   0.2738302  -0.04273592]
[ 0.21681469  0.27379223 -0.04273558]
[ 0.21688787  0.27375404 -0.04273521]
[ 0.216961    0.2737177  -0.04272909]
[ 0.21704014  0.27367915 -0.04272867]
[ 0.21701646  0.27364218 -0.04272867]
[ 0.21710757  0.27360322 -0.04272818]
[ 0.21718052  0.27356443 -0.04272775]
[ 0.21724132  0.27352375 -0.04273309]
[ 0.21730207  0.27348493 -0.0427327 ]
[ 0.21727814  0.27344758 -0.0427327 ]
[ 0.21733274  0.27340869 -0.04273232]
[ 0.21736313  0.27337212 -0.04272633]
[ 0.2173874   0.27333361 -0.04272609]
[ 0.21741165  0.27329501 -0.04272584]
[ 0.21738748  0.27325728 -0.04272584]
[ 0.21740558  0.27321863 -0.04272563]
[ 0.21741757  0.27318002 -0.04272544]
[ 0.21743557  0.27313923 -0.04273095]
[ 0.21745956

In [ ]:
print(measured_finger_position[0])

In [ ]:
def plot_finger_trajectory(finger_position, ball_positions):
    """
    plots the position of the finger in 2D and the position of the spatial frame {s}
    we assume that the time varying x variable is in x_pos and that the y variable is in y_pos
    """
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    plt.plot(finger_position[:,0],finger_position[:,1], 'b')
    plt.xlabel('finger x position [m]')
    plt.ylabel('finger y position [m]')
    plt.xlim([-l1-l2+l0-0.05,l0+l1+l2+0.05])
    plt.ylim([-l1-l2-0.05, l1+l2+0.05])
    plt.plot([0],[0],'o',markersize=15,color='r')
    
    for ball in ball_positions:
        plt.plot([ball[0]],[ball[1]],'o',markersize=15,color='g')
        ax.annotate('Goal position', xy=(ball[0]-0.2,0.05+ball[1]), xytext=(30,0), textcoords='offset points')
    ax.annotate('Spatial frame {s}', xy=(-0.03,-0.005), xytext=(30,0), textcoords='offset points')
    
    fig = plt.figure()
    
    plt.subplot(3,1,1)
    plt.plot(time, measured_finger_position[:,0], 'b-', time, desired_finger_position[:,0], '--k')
    plt.ylabel(r'$x$')
    plt.subplot(3,1,2)
    plt.plot(time, measured_finger_position[:,1], 'b-', time, desired_finger_position[:,1], '--k')
    plt.ylabel(r'$y$')
    plt.subplot(3,1,3)
    plt.plot(time, measured_finger_position[:,2], 'b-', time, desired_finger_position[:,2], '--k')
    plt.ylabel(r'$z$')
    plt.xlabel('Time [s]')

plot_finger_trajectory(measured_finger_position, ball_positions)

In [ ]:
def plot_joint_posvel(time, th, th_des, dth, dth_des):
    fig = plt.figure(figsize=(10,10))
    
    plt.subplot(3,2,1)
    plt.plot(time, th[:,0], 'b-', time, th_des[:,0], '--k')
    plt.ylabel(r'$\theta_0$')
    plt.subplot(3,2,3)
    plt.plot(time, th[:,1], 'b-', time, th_des[:,1], '--k')
    plt.ylabel(r'$\theta_1$')
    plt.subplot(3,2,5)
    plt.plot(time, th[:,2], 'b-', time, th_des[:,2], '--k')
    plt.ylabel(r'$\theta_2$')
    plt.xlabel('Time [s]')
    
    plt.subplot(3,2,2)
    plt.plot(time, dth[:,0], 'b-', time, dth_des[:,0], '--k')
    plt.ylabel(r'$\dot{\theta}_0$')
    plt.subplot(3,2,4)
    plt.plot(time, dth[:,1], 'b-', time, dth_des[:,1], '--k')
    plt.ylabel(r'$\dot{\theta}_1$')
    plt.subplot(3,2,6)
    plt.plot(time, dth[:,2], 'b-', time, dth_des[:,2], '--k')
    plt.ylabel(r'$\dot{\theta}_2$')
    plt.xlabel('Time [s]')
    
plot_joint_posvel(time, measured_positions, desired_positions, measured_velocities, desired_velocities)